In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import tensorflow
import keras

Using TensorFlow backend.


In [2]:
import struct

class WavFileHelper():
    
    def read_file_properties(self, filename):

        wave_file = open(filename,"rb")
        
        riff = wave_file.read(12)
        fmt = wave_file.read(36)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I",sample_rate_string)[0]
        
        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H",bit_depth_string)[0]

        return (num_channels, sample_rate, bit_depth)
wavfilehelper = WavFileHelper()

In [3]:
audiodata = []
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')


for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath('G:/urban_dataset/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    data = wavfilehelper.read_file_properties(file_name)
    audiodata.append(data)

# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['num_channels','sample_rate','bit_depth'])

In [6]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') # default is Kaiser_best
        stft = np.abs(librosa.stft(audio))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return chroma

In [7]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')

features = []
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

C:\Users\Admin\Anaconda3\envs\tensorflow\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Finished feature extraction from  8732  files


In [8]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [9]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

In [10]:
# neural network dimensions
n_dim =x_train.shape[1]
n_classes = y_train.shape[1]
n_hidden_units_1 = n_dim
n_hidden_units_2 = 400 # approx n_dim * 2
n_hidden_units_3 = 200 # half of layer 2

In [11]:
print(n_dim)
print("Features: {} Classes: {} ".format(n_dim, n_classes))

12
Features: 12 Classes: 10 


In [12]:
def create_model(activation_function='relu',init_type='normal', kernel_initializer="uniform", optimiser='Adamax', dropout_rate=0.5):
    model = Sequential()
    # layer 1
    #model.add(Convolution2D(n_hidden_units_1, input_dim=n_dim, activation=activation_function,kernel_initializer="uniform"))
    #model.add(MaxPooling2D(2,2))
    model.add(Dense(n_hidden_units_1, input_dim=n_dim, init=init_type, activation=activation_function))
    # layer 2
    model.add(Dense(n_hidden_units_2, kernel_initializer="uniform", activation=activation_function))
    model.add(Dropout(dropout_rate))
    # layer 3
    model.add(Dense(n_hidden_units_3, kernel_initializer="uniform" , activation=activation_function))
    model.add(Dropout(dropout_rate))
    # output layer
    model.add(Dense(n_classes, kernel_initializer="uniform", activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimiser, metrics=['accuracy'])
    return model

In [13]:
model = create_model()
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

C:\Users\Admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=12, activation="relu", kernel_initializer="normal")`
  


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 400)               5200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 87,566
Trainable params: 87,566
Non-trainable params: 0
__________________________________________________

In [15]:
history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, callbacks=[earlystop], batch_size=24)

Train on 6286 samples, validate on 699 samples
Epoch 1/30
6286/6286 [==============================] - 3s 441us/step - loss: 2.2569 - acc: 0.1102 - val_loss: 2.2282 - val_acc: 0.1359
Epoch 2/30
6286/6286 [==============================] - 2s 385us/step - loss: 2.1932 - acc: 0.1729 - val_loss: 2.0916 - val_acc: 0.2203
Epoch 3/30
6286/6286 [==============================] - 2s 346us/step - loss: 2.0535 - acc: 0.2014 - val_loss: 1.9799 - val_acc: 0.2017
Epoch 4/30
6286/6286 [==============================] - 2s 370us/step - loss: 2.0021 - acc: 0.2119 - val_loss: 1.9608 - val_acc: 0.2375
Epoch 5/30
6286/6286 [==============================] - 2s 368us/step - loss: 1.9744 - acc: 0.2171 - val_loss: 2.0046 - val_acc: 0.2031
Epoch 6/30
6286/6286 [==============================] - 2s 372us/step - loss: 1.9695 - acc: 0.2133 - val_loss: 1.9376 - val_acc: 0.2418
Epoch 7/30
6286/6286 [==============================] - 2s 387us/step - loss: 1.9565 - acc: 0.2211 - val_loss: 1.9241 - val_acc: 0.2589
E

In [16]:
score, accuracy = model.evaluate(x = x_test, y = y_test , verbose = 1)
print("ACCURACY:",(accuracy*100))
print("SCORE:",score)

1747/1747 [==============================] - 0s 134us/step
ACCURACY: 30.68116771779063
SCORE: 1.7754048659723694


In [17]:
model.save("nn_chroma_model.h5")
print("Saved model to disk")

Saved model to disk


In [18]:
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('nn_chroma_model.h5')
# summarize model.
model.summary()
# load dataset

# evaluate the model
score = model.evaluate(x=x_test,y=y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 400)               5200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 87,566
Trainable params: 87,566
Non-trainable params: 0
__________________________________________________